# Test VQE Algorithm

In [1]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
target_distr /= sum(target_distr)

In [2]:
target_distr

array([0.51357006, 0.48642994])

In [5]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name="c")
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [7]:
# objective func
from qiskit import Aer, transpile, assemble
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the parameters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

In [13]:
# This version is old, replace it.
# create optimizer
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
# from qiskit.aqua.components.optimizers import COBYLA
from qiskit.algorithms.optimizers import COBYLA

optimizer = COBYLA(maxiter=500, tol=0.0001)
params = np.random.rand(3)

# ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# ret = optimizer.minimize(num_vars=3, objective_function=objective_function, initial_point=params)

# obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])


TypeError: minimize() got an unexpected keyword argument 'num_vars'

In [15]:
# using new version
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import VQE

optimizer = COBYLA(maxiter=500, tol=0.0001)
params = np.random.rand(3)
qc = get_var_form(params)

vqe = VQE(optimizer=COBYLA(), quantum_instance=qc)
# ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# ret = optimizer.minimize(num_vars=3, objective_function=objective_function, initial_point=params)

# obtain the output distribution using the final parameters

t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
# print("Output Error (Manhattan Distance):", ret[1])
# print("Parameters Found:", ret[0])


TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [14]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.21.1', 'qiskit-aer': '0.10.4', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.37.1', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [1]:
import pytket
print(pytket.__version__)

1.5.0


# New Version VQE Code

In [16]:
from qiskit import Aer
from qiskit.opflow import X, Z, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

H2_op = (-1.052373245772859 * I ^ I) + \
        (0.39793742484318045 * I ^ Z) + \
        (-0.39793742484318045 * Z ^ I) + \
        (-0.01128010425623538 * Z ^ Z) + \
        (0.18093119978423156 * X ^ X)

In [17]:
print(H2_op)

-1.052373245772859 * II
+ 0.39793742484318045 * IZ
- 0.39793742484318045 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423156 * XX


In [18]:
seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)
optimizer_evals = result.optimizer_evals

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 65,
    'eigenstate': array([ 9.55146158e-05+0.j, -9.93766272e-01+0.j,  1.11483575e-01+0.j,
        1.77493990e-05+0.j]),
    'eigenvalue': (-1.8572750175664259+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[6]): -4.717618171283927,
                              ParameterVectorElement(θ[7]): 0.3602072577510429,
                              ParameterVectorElement(θ[4]): -2.5983258978150006,
                              ParameterVectorElement(θ[2]): 0.5470754664946292,
                              ParameterVectorElement(θ[3]): 6.092947779034692,
                              ParameterVectorElement(θ[5]): 1.5683260003556614,
                              ParameterVectorElement(θ[1]): 4.426962083985579,
                              ParameterVectorElement(θ[0]): 4.296520455019831},
    'optimal_point': array([ 4.29652046,  4.42696208,  0.54707547,  6.09294778, -2.5983259 ,
        1.568326  , -4.71761817,  0.3

In [19]:
initial_pt = result.optimal_point

algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, initial_point=initial_pt, quantum_instance=qi)
result1 = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result1)
optimizer_evals1 = result1.optimizer_evals
print()
print(f'optimizer_evals is {optimizer_evals1} with initial point versus {optimizer_evals} without it.')

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 9,
    'eigenstate': array([ 9.55146158e-05+0.j, -9.93766272e-01+0.j,  1.11483575e-01+0.j,
        1.77493990e-05+0.j]),
    'eigenvalue': (-1.8572750175664259+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[7]): 0.3602072577510429,
                              ParameterVectorElement(θ[1]): 4.426962083985579,
                              ParameterVectorElement(θ[4]): -2.5983258978150006,
                              ParameterVectorElement(θ[2]): 0.5470754664946292,
                              ParameterVectorElement(θ[3]): 6.092947779034692,
                              ParameterVectorElement(θ[0]): 4.296520455019831,
                              ParameterVectorElement(θ[6]): -4.717618171283927,
                              ParameterVectorElement(θ[5]): 1.5683260003556614},
    'optimal_point': array([ 4.29652046,  4.42696208,  0.54707547,  6.09294778, -2.5983259 ,
        1.568326  , -4.71761817,  0.36

In [22]:
# compute the Expectation value
seed

50

In [23]:
from qiskit import Aer

algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('aer_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi, include_custom=True)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
optimal_value1 = result.optimal_value
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 65,
    'eigenstate': {'01': 0.9921567416492215, '10': 0.125},
    'eigenvalue': (-1.8572750175571595+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[0]): 4.2965205340503685,
                              ParameterVectorElement(θ[1]): 4.426962242132452,
                              ParameterVectorElement(θ[5]): 1.5683259454122547,
                              ParameterVectorElement(θ[4]): -2.5983258639687397,
                              ParameterVectorElement(θ[3]): 6.092947857528147,
                              ParameterVectorElement(θ[7]): 0.36020735708081203,
                              ParameterVectorElement(θ[2]): 0.5470754193210003,
                              ParameterVectorElement(θ[6]): -4.717618177195121},
    'optimal_point': array([ 4.29652053,  4.42696224,  0.54707542,  6.09294786, -2.59832586,
        1.56832595, -4.71761818,  0.36020736]),
    'optimal_value': -1.8572750175571595,
    

In [24]:
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('aer_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
optimal_value = result.optimal_value
print('The optimal value can be seen to be wrong too, i.e. '
      f'{optimal_value:.3f} versus the correct {optimal_value1:.3f}.')
print()
print(result)

The optimal value can be seen to be wrong too, i.e. -1.099 versus the correct -1.857.

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 9,
    'eigenstate': {   '00': 0.7781187248742958,
                      '01': 0.4881406047441659,
                      '10': 0.39404750665370286,
                      '11': 0.03125},
    'eigenvalue': (-1.0987888676631705+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[2]): 0.6019852007557844,
                              ParameterVectorElement(θ[1]): 4.19301252102391,
                              ParameterVectorElement(θ[3]): 5.949536809130025,
                              ParameterVectorElement(θ[6]): -5.466043598406607,
                              ParameterVectorElement(θ[7]): 0.6984088030463615,
                              ParameterVectorElement(θ[4]): -3.3070470445355764,
                              ParameterVectorElement(θ[5]): 1.8462931831829383,
                              ParameterVectorElement(θ[0]): 